# Use AI21's Azure Client with Jamba 1.5 Large and Jamba 1.5 Mini through Azure AI Models-as-a-Service

Use AI21's Azure client to consume Jamba 1.5 Large or Jamba 1.5 Mini deployments in Azure AI and Azure ML through serverless API endpoints delivered through Models-as-a-Service (MaaS).

> Review the documentation for Jamba 1.5 models for [AI Studio](https://aka.ms/ai21-jamba-1.5-large-ai-studio-docs) and for [ML Studio](https://aka.ms/ai21-jamba-1.5-large-ml-studio-docs) for details on how to provision inference endpoints, regional availability, pricing and inference schema reference.

The below samples are seen on [AI21's GitHub](https://github.com/AI21Labs/ai21-python/tree/main/examples/studio) and shared here for ease of use with their Azure client.

## Prerequisites

Before we start, there are certain steps we need to take to deploy the models:

* Register for a valid Azure account with subscription 
* Make sure you have access to [Azure AI Studio](https://learn.microsoft.com/en-us/azure/ai-studio/what-is-ai-studio?tabs=home)
* Create a project and resource group
* Select `AI21 Jamba 1.5 Large` or `AI21 Jamba 1.5 Mini`

    > Notice that some models may not be available in all the regions in Azure AI and Azure Machine Learning. On those cases, you can create a workspace or project in the region where the models are available and then consume it with a connection from a different one. To learn more about using connections see [Consume models with connections](https://learn.microsoft.com/en-us/azure/ai-studio/how-to/deployments-connections)

* Deploy with "Pay-as-you-go"

Once deployed successfully, you should be assigned for an API endpoint and a security key for inference.

For more information, you should consult Azure's official documentation [here](https://aka.ms/ai21-jamba-1.5-large-azure-ai-studio-docs) for model deployment and inference.

To complete this tutorial, you will need to: 

* Install `ai21`:

    ```bash
    pip install -U "ai21>=2.13.0"
    ```
* If it's not working on your first go, try restarting the kernel and then run the pip install again.

## General Example

The following is an example about how to use `ai21`'s client on Azure and leveraging this for AI21 Jamba 1.5 Large through MaaS.

In [ ]:
%pip install -U "ai21>=2.13.0"

In [ ]:
%pip install asyncio aiohttp

In [ ]:
import os
import json
import uuid
from enum import Enum
import asyncio
from pydantic import BaseModel
from ai21 import AsyncAI21Client
from ai21 import AI21AzureClient
from ai21.models.chat import (
    ChatMessage,
    ResponseFormat,
    ToolMessage,
    FunctionToolDefinition,
    DocumentSchema,
)
from ai21.models.chat.chat_message import SystemMessage, UserMessage, AssistantMessage
from ai21.models.chat.function_tool_definition import FunctionToolDefinition
from ai21.models.chat.tool_defintions import ToolDefinition
from ai21.models.chat.tool_parameters import ToolParameters

To use `ai21`, create a client and configure it as follows:

- `endpoint`: Use the endpoint URL from your deployment. Include `/v1` at the end of the endpoint.
- `api_key`: Use your API key.

In [ ]:
base_url = "<your-maas-endpoint>"
api_key = "<your-api-key>"

In [ ]:
client = AI21AzureClient(base_url=base_url, api_key=api_key)
# async_client = AsyncAI21Client(base_url=base_url, api_key=api_key)

In [ ]:
model = (
    "jamba-1.5-large"  # Change to "jamba-1.5-mini" if you'd like to try the Mini model
)

Alternatively, you can set an environment variable for your API key:

In [ ]:
import os

os.environ["AI21_API_KEY"] = "<your-api-key>"
client = AI21AzureClient(
    base_url="<your-maas-endpoint>", api_key=os.environ.get("AI21_API_KEY")
)

Use the client to create chat completions requests:

In [ ]:
system = "You're a support engineer in a SaaS company"

messages = [
    SystemMessage(content=system, role="system"),
    UserMessage(content="Hello, I need help with a signup process.", role="user"),
    AssistantMessage(
        content="Hi Alice, I can help you with that. What seems to be the problem?",
        role="assistant",
    ),
    UserMessage(
        content="I am having trouble signing up for your product with my Google account.",
        role="user",
    ),
]

chat_completions = client.chat.completions.create(
    model=model,
    messages=messages,
    temperature=1.0,  # Setting =1 allows for greater variability per API call.
    top_p=1.0,  # Setting =1 allows full sample of tokens to be considered per API call.
    max_tokens=100,
    stop=["\n"],
)

The generated text can be accessed as follows:

In [ ]:
print(chat_completions.to_json())

### Incorporating Chat response formatting

In [ ]:
class TicketType(Enum):
    ADULT = "adult"
    CHILD = "child"


class ZooTicket(BaseModel):
    ticket_type: TicketType
    quantity: int


class ZooTicketsOrder(BaseModel):
    date: str
    tickets: list[ZooTicket]


messages = [
    ChatMessage(
        role="user",
        content="Please create a JSON object for ordering zoo tickets for September 22, 2024, "
        f"for myself and two kids, based on the following JSON schema: {ZooTicketsOrder.schema()}.",
    )
]

response = client.chat.completions.create(
    messages=messages,
    model=model,
    max_tokens=800,
    temperature=0,
    response_format=ResponseFormat(type="json_object"),
)

zoo_order_json = json.loads(response.choices[0].message.content)
print(zoo_order_json)

### Chat Function calling

In [ ]:
def get_order_delivery_date(order_id: str) -> str:
    print(f"Retrieving the delivery date for order ID: {order_id} from the database...")
    return "2025-05-04"


messages = [
    ChatMessage(
        role="system",
        content="You are a helpful customer support assistant. Use the supplied tools to assist the user.",
    ),
    ChatMessage(
        role="user", content="Hi, can you tell me the delivery date for my order?"
    ),
    ChatMessage(
        role="assistant",
        content="Hi there! I can help with that. Can you please provide your order ID?",
    ),
    ChatMessage(role="user", content="i think it is order_12345"),
]

tool_definition = ToolDefinition(
    type="function",
    function=FunctionToolDefinition(
        name="get_order_delivery_date",
        description="Retrieve the delivery date associated with the specified order ID",
        parameters=ToolParameters(
            type="object",
            properties={
                "order_id": {
                    "type": "string",
                    "description": "The customer's order ID.",
                }
            },
            required=["order_id"],
        ),
    ),
)

tools = [tool_definition]

response = client.chat.completions.create(
    messages=messages, model="jamba-1.5-large", tools=tools
)

""" AI models can be error-prone, it's crucial to ensure that the tool calls align with the expectations.
The below code snippet demonstrates how to handle tool calls in the response and invoke the tool function
to get the delivery date for the user's order. After retrieving the delivery date, we pass the response back
to the AI model to continue the conversation, using the ToolMessage object. """

assistant_message = response.choices[0].message
messages.append(assistant_message)  # Adding the assistant message to the chat history

delivery_date = None
tool_calls = assistant_message.tool_calls
if tool_calls:
    tool_call = tool_calls[0]
    if tool_call.function.name == "get_order_delivery_date":
        func_arguments = tool_call.function.arguments
        func_args_dict = json.loads(func_arguments)

        if "order_id" in func_args_dict:
            delivery_date = get_order_delivery_date(func_args_dict["order_id"])
        else:
            print("order_id not found in function arguments")
    else:
        print(f"Unexpected tool call found - {tool_call.function.name}")
else:
    print("No tool calls found")

if delivery_date is not None:
    """Continue the conversation by passing the delivery date back to the model"""

    tool_message = ToolMessage(
        role="tool", tool_call_id=tool_calls[0].id, content=delivery_date
    )
    messages.append(tool_message)

    response = client.chat.completions.create(
        messages=messages, model="jamba-1.5-large", tools=tools
    )
    print(response.choices[0].message.content)

### Chat with Document Schema 

In [ ]:
schnoodel = DocumentSchema(
    id=str(uuid.uuid4()),
    content="Schnoodel Inc. Annual Report - 2024. Schnoodel Inc., a leader in innovative culinary technology, saw a "
    "15% revenue growth this year, reaching $120 million. The launch of SchnoodelChef Pro has significantly "
    "contributed, making up 35% of total sales. We've expanded into the Asian market, notably Japan, "
    "and increased our global presence. Committed to sustainability, we reduced our carbon footprint "
    "by 20%. Looking ahead, we plan to integrate more advanced machine learning features and expand "
    "into South America.",
    metadata={"topic": "revenue"},
)
shnokel = DocumentSchema(
    id=str(uuid.uuid4()),
    content="Shnokel Corp. Annual Report - 2024. Shnokel Corp., a pioneer in renewable energy solutions, "
    "reported a 20% increase in revenue this year, reaching $200 million. The successful deployment of "
    "our advanced solar panels, SolarFlex, accounted for 40% of our sales. We entered new markets in Europe "
    "and have plans to develop wind energy projects next year. Our commitment to reducing environmental "
    "impact saw a 25% decrease in operational emissions. Upcoming initiatives include a significant "
    "investment in R&D for sustainable technologies.",
    metadata={"topic": "revenue"},
)

documents = [schnoodel, shnokel]

messages = [
    ChatMessage(
        role="system",
        content="You are a helpful assistant that receives revenue documents and answers related questions",
    ),
    ChatMessage(
        role="user",
        content="Hi, which company earned more during 2024 - Schnoodel or Shnokel?",
    ),
]

response = client.chat.completions.create(
    messages=messages, model="jamba-1.5-mini", documents=documents
)

print(response)

### Stream Chat Completions

In [ ]:
system = "You're a support engineer in a SaaS company"
messages = [
    ChatMessage(content=system, role="system"),
    ChatMessage(content="Hello, I need help with a signup process.", role="user"),
    ChatMessage(
        content="Hi Alice, I can help you with that. What seems to be the problem?",
        role="assistant",
    ),
    ChatMessage(
        content="I am having trouble signing up for your product with my Google account.",
        role="user",
    ),
]

response = client.chat.completions.create(
    messages=messages,
    model=model,
    max_tokens=100,
    stream=True,
)
for chunk in response:
    print(chunk.choices[0].delta.content, end="")

### Chat function calling with multiple tools

In [ ]:
def get_weather(place: str, date: str) -> str:
    """
    Retrieve the expected weather for a specified location and date.
    """
    print(f"Fetching expected weather for {place} on {date}...")
    return "32 celsius"


def get_sunset_hour(place: str, date: str) -> str:
    """
    Fetch the expected sunset time for a given location and date.
    """
    print(f"Fetching expected sunset time for {place} on {date}...")
    return "7 pm"


messages = [
    ChatMessage(
        role="system",
        content="You are a helpful assistant. Use the supplied tools to assist the user.",
    ),
    ChatMessage(
        role="user",
        content="Hello, could you help me find out the weather forecast and sunset time for London?",
    ),
    ChatMessage(
        role="assistant", content="Hi there! I can help with that. On which date?"
    ),
    ChatMessage(role="user", content="At 2024-08-27"),
]

get_sunset_tool = ToolDefinition(
    type="function",
    function=FunctionToolDefinition(
        name="get_sunset_hour",
        description="Fetch the expected sunset time for a given location and date.",
        parameters=ToolParameters(
            type="object",
            properties={
                "place": {
                    "type": "string",
                    "description": "The location for which the weather is being queried.",
                },
                "date": {
                    "type": "string",
                    "description": "The date for which the weather is being queried.",
                },
            },
            required=["place", "date"],
        ),
    ),
)

get_weather_tool = ToolDefinition(
    type="function",
    function=FunctionToolDefinition(
        name="get_weather",
        description="Retrieve the expected weather for a specified location and date.",
        parameters=ToolParameters(
            type="object",
            properties={
                "place": {
                    "type": "string",
                    "description": "The location for which the weather is being queried.",
                },
                "date": {
                    "type": "string",
                    "description": "The date for which the weather is being queried.",
                },
            },
            required=["place", "date"],
        ),
    ),
)

tools = [get_sunset_tool, get_weather_tool]

response = client.chat.completions.create(
    messages=messages, model="jamba-1.5-large", tools=tools
)

""" AI models can be error-prone, it's crucial to ensure that the tool calls align with the expectations.
The below code snippet demonstrates how to handle tool calls in the response and invoke the tool function
to get the delivery date for the user's order. After retrieving the delivery date, we pass the response back
to the AI model to continue the conversation, using the ToolMessage object. """

assistant_message = response.choices[0].message
messages.append(assistant_message)  # Adding the assistant message to the chat history

too_call_id_to_result = {}
tool_calls = assistant_message.tool_calls
if tool_calls:
    for tool_call in tool_calls:
        if tool_call.function.name == "get_weather":
            """Verify get_weather tool call arguments and invoke the function to get the weather forecast:"""
            func_arguments = tool_call.function.arguments
            args = json.loads(func_arguments)

            if "place" in args and "date" in args:
                result = get_weather(args["place"], args["date"])
                too_call_id_to_result[tool_call.id] = result
            else:
                print(f"Got unexpected arguments in function call - {args}")

        elif tool_call.function.name == "get_sunset_hour":
            """Verify get_sunset_hour tool call arguments and invoke the function to get the weather forecast:"""
            func_arguments = tool_call.function.arguments
            args = json.loads(func_arguments)

            if "place" in args and "date" in args:
                result = get_sunset_hour(args["place"], args["date"])
                too_call_id_to_result[tool_call.id] = result
            else:
                print(f"Got unexpected arguments in function call - {args}")

        else:
            print(f"Unexpected tool call found - {tool_call.function.name}")
else:
    print("No tool calls found")

if too_call_id_to_result:
    """Continue the conversation by passing the sunset and weather back to the AI model:"""

    for tool_id_called, result in too_call_id_to_result.items():
        tool_message = ToolMessage(
            role="tool", tool_call_id=tool_id_called, content=str(result)
        )
        messages.append(tool_message)

    response = client.chat.completions.create(
        messages=messages, model="jamba-1.5-large", tools=tools
    )
    print(response.choices[0].message.content)

### Async Stream Chat Completions

In [ ]:
import asyncio

from ai21 import AsyncAI21Client, AsyncAI21AzureClient
from ai21.models.chat import ChatMessage

system = "You're a support engineer in a SaaS company"
messages = [
    ChatMessage(content=system, role="system"),
    ChatMessage(content="Hello, I need help with a signup process.", role="user"),
    ChatMessage(
        content="Hi Alice, I can help you with that. What seems to be the problem?",
        role="assistant",
    ),
    ChatMessage(
        content="I am having trouble signing up for your product with my Google account.",
        role="user",
    ),
]

client = AsyncAI21AzureClient(base_url=base_url, api_key=api_key)


async def main():
    response = await client.chat.completions.create(
        messages=messages,
        model=model,
        max_tokens=100,
        stream=True,
    )
    async for chunk in response:
        print(chunk.choices[0].delta.content, end="")


loop = asyncio.get_event_loop()
loop.create_task(main())

### Asynch Chat Completions

In [ ]:
import asyncio

from ai21 import AsyncAI21AzureClient
from ai21.models.chat import ChatMessage


system = "You're a support engineer in a SaaS company"
messages = [
    ChatMessage(content=system, role="system"),
    ChatMessage(content="Hello, I need help with a signup process.", role="user"),
    ChatMessage(
        content="Hi Alice, I can help you with that. What seems to be the problem?",
        role="assistant",
    ),
    ChatMessage(
        content="I am having trouble signing up for your product with my Google account.",
        role="user",
    ),
]

client = AsyncAI21AzureClient(base_url=base_url, api_key=api_key)


async def main():
    response = await client.chat.completions.create(
        messages=messages,
        model=model,
        max_tokens=100,
        temperature=0.7,
        top_p=1.0,
        stop=["\n"],
    )

    print(response)


loop = asyncio.get_event_loop()
loop.create_task(main())

## Aditional resources

Here are some additional reference:  

* [Plan and manage costs (marketplace)](https://learn.microsoft.com/azure/ai-studio/how-to/costs-plan-manage#monitor-costs-for-models-offered-through-the-azure-marketplace)